In [3]:
# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2

In [4]:
#pip install imutils
from playsound import playsound

In [7]:
def eye_aspect_ratio(eye):
# compute the euclidean distances between the two sets of
# vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
# compute the euclidean distance between the horizontal
# eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
# compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
# return the eye aspect ratio
    return ear


def calculate_MAR(mouth):
    A = dist.euclidean(mouth[2], mouth[6])
    B = dist.euclidean(mouth[3], mouth[5])
    C= dist.euclidean(mouth[1], mouth[7])
    D=dist.euclidean(mouth[0], mouth[4])
    mar= (A+B+C)/(3*D)
    
    return mar

def sound_alarm():
	# play an alarm sound
#Replace it with the sound you want to use
    playsound("C:\\Users\\asaifbabu\\Downloads\\beep-02.mp3")

In [8]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart,mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["inner_mouth"]

ear_thres=0.20
no_of_frames=20
drowsy=False
counter=0
print("[INFO] loading facial landmark predictor...")
#detector = dlib.get_frontal_face_detector()

camera = cv2.VideoCapture(0)
face= dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:\\Users\\asaifbabu\\Downloads\\shape_predictor_68_face_landmarks.dat\\shape_predictor_68_face_landmarks.dat")
while True:
    # Capture frame-by-frame
    ret, frame = camera.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #detectFace
    faces=face(gray)
    #Process on detected face
    for i in faces:
        shape = predictor(gray, i)
        shape = face_utils.shape_to_np(shape)
        
        x1=i.left()
        y1=i.top()
        x2=i.right()
        y2=i.bottom()
        roi_gray = gray[y1:y2, x1:x2]
        roi_color = frame[y1:y2, x1:x2]
    cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
    
    #Get cordinates data for eyes and mouth
    left_eye=shape[lStart:lEnd]
    right_eye=shape[rStart:rEnd]
    mouth=shape[mStart:mEnd]
    
    #Draw on Eyes
    leftEyeHull = cv2.convexHull(left_eye)
    rightEyeHull = cv2.convexHull(right_eye)
    mouthHull=cv2.convexHull(mouth)
    cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
    cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
    #cv2.drawContours(frame, [mouth], -1, (0, 255, 0), 1)
    #cv2.putText(frame, "6", (mouth[6][0],mouth[6][1]),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    #cv2.putText(frame, "2", (mouth[2][0],mouth[2][1]),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    #cv2.putText(frame, "3", (mouth[3][0],mouth[3][1]),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    #cv2.putText(frame, "5", (mouth[5][0],mouth[5][1]),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    #cv2.putText(frame, "0", (mouth[0][0],mouth[0][1]),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    #cv2.putText(frame, "7", (mouth[7][0],mouth[7][1]),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    #cv2.putText(frame, "1", (mouth[1][0],mouth[1][1]),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    #cv2.putText(frame, "4", (mouth[4][0],mouth[4][1]),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    left_ear=eye_aspect_ratio(left_eye)
    right_ear=eye_aspect_ratio(right_eye)
    #Avg EAR of both Eyes
    EAR=(left_ear+right_ear)/2
    mar= calculate_MAR(mouth)
    mar_thres= ear_thres+0.17
    #print(mar)
    if EAR <ear_thres:
        counter+=1
        if counter>=no_of_frames:
            drowsy=True
            # draw an alarm on the frame
            
    elif mar>mar_thres:
        counter+=1
        if counter>=no_of_frames:
            drowsy=True
            # draw an alarm on the frame
            
    else:
        drowsy=False
        counter=0
        cv2.putText(frame, "ALERT", (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, "EAR: {:.2f}".format(EAR), (200, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, "MAR: {:.2f}".format(mar), (350, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    if(drowsy==True):
        cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        t = Thread(target=sound_alarm(),)
        t.deamon = True
        t.start()

    cv2.imshow('OUTPUT',frame)
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
camera.release()
cv2.destroyAllWindows()



[INFO] loading facial landmark predictor...
